In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
x = torch.rand(2, 20)
net(x)

tensor([[-0.0522,  0.0948, -0.0159, -0.0558,  0.0991, -0.0595, -0.2030,  0.0051,
         -0.0843,  0.1441],
        [-0.0256,  0.2283, -0.0608, -0.0135,  0.0392, -0.0968, -0.3012,  0.0817,
         -0.1332,  0.0629]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net.forward(x)

tensor([[-0.0349,  0.0687, -0.0080,  0.0244,  0.0476, -0.1822,  0.1992,  0.0818,
         -0.2425, -0.2073],
        [-0.1301,  0.0603, -0.0659,  0.0373,  0.1093,  0.0076,  0.0553,  0.0057,
         -0.1421, -0.2748]], grad_fn=<AddmmBackward0>)

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net.forward(x)

tensor([[-0.0184, -0.1486, -0.1323,  0.0667,  0.1622,  0.1296, -0.0553,  0.2700,
          0.2256, -0.1512],
        [-0.0077, -0.0934, -0.0644, -0.0414,  0.2293, -0.0290, -0.1314,  0.1088,
          0.1430, -0.2465]], grad_fn=<AddmmBackward0>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
            X = self.linear(X)
            # 使用创建的常量参数以及relu和mm函数
            X = F.relu(torch.mm(X, self.rand_weight) + 1)
            # 复用全连接层。这相当于两个全连接层共享参数
            X = self.linear(X)
            # 控制流
            while X.abs().sum() > 1:
                X /= 2
            return X.sum()

In [11]:
net = FixedHiddenMLP()
net(x)

tensor(0.3116, grad_fn=<SumBackward0>)

In [13]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(x)

tensor(0.1148, grad_fn=<SumBackward0>)